In [1]:
import pandas as pd
import numpy as np
import holoviews as hv

from functools import wraps
import logging

def log_shape(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        logging.info("%s,%s" % (func.__name__, result.shape))
        return result
    return wrapper

def log_dtypes(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        logging.info("%s,%s" % (func.__name__, result.dtypes))
        return result
    return wrapper

In [4]:
prod_df = pd.read_csv('./billa_shop_products.csv')
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10123 entries, 0 to 10122
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   articleId  10123 non-null  object 
 1   name       10123 non-null  object 
 2   price      10123 non-null  float64
 3   brand      9549 non-null   object 
 4   group      10123 non-null  object 
dtypes: float64(1), object(4)
memory usage: 395.6+ KB


In [8]:
prod_df['group'].unique()

array(['Obst & Gemüse', 'Brot & Gebäck', 'Getränke', 'Kühlwaren',
       'Tiefkühl', 'Grundnahrungsmittel', 'Süßes & Salziges', 'Pflege',
       'Haushalt', 'Haustier', 'Geschenkideen'], dtype=object)

In [10]:
prod_df.groupby('group').size().sort_values(ascending=False)

group
Grundnahrungsmittel    2659
Kühlwaren              1789
Getränke               1680
Süßes & Salziges        905
Tiefkühl                738
Pflege                  641
Haushalt                605
Brot & Gebäck           435
Haustier                340
Obst & Gemüse           278
Geschenkideen            53
dtype: int64

In [13]:
prod_df.groupby('group')['price'].describe().sort_values('mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
group,,,,,,,,
Geschenkideen,53.0,16.649811,16.743175,1.89,4.29,8.49,21.99,61.99
Getränke,1680.0,6.148911,8.109358,0.27,1.59,3.29,7.49,61.99
Kühlwaren,1789.0,5.395394,7.409653,0.29,1.59,2.59,4.19,66.99
Tiefkühl,738.0,4.810976,2.997192,0.69,3.29,3.99,5.49,19.99
Pflege,641.0,4.772059,3.629718,0.49,2.59,3.89,5.49,29.99
Haushalt,605.0,4.311140,3.095568,0.59,1.99,3.49,5.49,24.99
Haustier,340.0,3.245088,3.594358,0.20,0.99,1.99,4.49,24.99
Brot & Gebäck,435.0,3.048276,2.724431,0.19,1.59,2.29,3.49,21.90
Obst & Gemüse,278.0,2.791978,1.456861,0.69,1.99,2.49,2.99,10.99


---

# Prod SQLite database

In [ ]:
import sqlite3

con = sqlite3.connect('./crawler/data/products_billa.sqlite3')

with con:
    query_reduced_products = '''
        SELECT prod_id, MIN(price_normal), MAX(price_normal)
        FROM prices
        GROUP BY prod_id
        HAVING (MAX(price_normal) - MIN(price_normal)) < 0
        '''
    res = con.execute(query_reduced_products).fetchall()
res